# Global sistem çözümü

Bu derste ders 7 kapsamında daha önce gördüğümüz tüm bilgileri kullanarak global olarak sistemin tamamını çözümleyeceğiz. Çözeceğimiz problemi tekrar göz önüne getirelim:

<img src="Resimler/Ders7/2BoyutluProblem.png" align="center" width= "600"/>

İlk olarak problemin parametrelerini belirleyerek başlayalım:

In [5]:
L = 2
H = 2
D = 0.05 #m

q_1 = -500  # W/m2
q_2 = -1000 # W/m2
q_3 = -1500 # W/m2

k = 5 # W/m2.K
T_sonsuz = 30 # C
h = 10 # W/m2K
h_z = 2*h/D

In [9]:
import sympy as sp
import numpy as np
from scipy import integrate
from sympy.utilities import lambdify

xi = sp.Symbol("xi")
eta = sp.Symbol("eta")
x_ = sp.symbols("x1,x2,x3,x4")
y_ = sp.symbols("y1,y2,y3,y4")
şekil = sp.Matrix([(1+xi)*(1+eta)/4,
                   (1-xi)*(1+eta)/4,
                   (1-xi)*(1-eta)/4,
                   (1+xi)*(1-eta)/4])

x_koord = [1,0,0,1] 
y_koord = [2,2,1,1]

def lokal_eleman_matrisi_olustur(x_koord, y_koord):
    nokta_sayısı = 4

    gradyen_xi = sp.diff(şekil, xi)
    gradyen_eta = sp.diff(şekil, eta)

    gradyen = sp.Matrix([gradyen_eta, gradyen_xi])

    gradyen = gradyen.reshape(2,4)

    c_şekil = sp.Matrix([0, 0])
    for i in range(nokta_sayısı):
        c_şekil[0] += x_[i] * şekil[i]
        c_şekil[1] += y_[i] * şekil[i]
    J = c_şekil.jacobian([xi, eta]).T
    
    for nokta in range(nokta_sayısı):
        J = J.subs(x_[nokta],x_koord[nokta]) 
        J = J.subs(y_[nokta],y_koord[nokta])
        
    K_e_sym = gradyen.T*(J**(-1)).T*k*(J**(-1))*gradyen*J.det()

    K_e = np.zeros((nokta_sayısı,nokta_sayısı))

    for i in range(nokta_sayısı):
        for j in range(nokta_sayısı):
            geçici_fonksiyon = lambdify( (xi,eta), K_e_sym[i,j], 'math' )
            K_e[i,j] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]
    
    K_t_sym = h_z*şekil*şekil.T*J.det()
    
    K_t = np.zeros((nokta_sayısı,nokta_sayısı))
    
    for i in range(nokta_sayısı):
        for j in range(nokta_sayısı):
            geçici_fonksiyon = lambdify( (xi,eta), K_t_sym[i,j], 'math' )
            K_t[i,j] += integrate.nquad(geçici_fonksiyon, [[-1, 1],[-1, 1]])[0]
    return K_e, K_t

K_e, K_t = lokal_eleman_matrisi_olustur(x_koord, y_koord)
K_e, K_t

(array([[ 3.33333333, -0.83333333, -1.66666667, -0.83333333],
        [-0.83333333,  3.33333333, -0.83333333, -1.66666667],
        [-1.66666667, -0.83333333,  3.33333333, -0.83333333],
        [-0.83333333, -1.66666667, -0.83333333,  3.33333333]]),
 array([[44.44444444, 22.22222222, 11.11111111, 22.22222222],
        [22.22222222, 44.44444444, 22.22222222, 11.11111111],
        [11.11111111, 22.22222222, 44.44444444, 22.22222222],
        [22.22222222, 11.11111111, 22.22222222, 44.44444444]]))

In [16]:
class FonksiyonUzayı():
    """
    Symbolic representation of the quadrilateral elements with four nodes
    """
    
    xi = sp.Symbol("xi")
    eta = sp.Symbol("eta")
    x_ = sp.symbols("x1,x2,x3,x4")
    y_ = sp.symbols("y1,y2,y3,y4")
    şekil = sp.Matrix([(1+xi)*(1+eta)/4,
                       (1-xi)*(1+eta)/4,
                       (1-xi)*(1-eta)/4,
                       (1+xi)*(1-eta)/4])
    nokta_sayısı = 4

    gradyen_xi = sp.diff(şekil, xi)
    gradyen_eta = sp.diff(şekil, eta)

    gradyen = sp.Matrix([gradyen_eta, gradyen_xi])

    gradyen = gradyen.reshape(2,4)

    c_şekil = sp.Matrix([0, 0])
    for i in range(nokta_sayısı):
        c_şekil[0] += x_[i] * şekil[i]
        c_şekil[1] += y_[i] * şekil[i]
    J = c_şekil.jacobian([xi, eta]).T

    def __init__(self, mesh):
        self.mesh = mesh
        self._dofmap = lambda e, r: self.mesh.hücreler[e, r]

    def dofmap(self):
        """
        Return dofmap for CG1 quad element, which is
        the same the local to global vertex number mapping of the mesh
        """
        return self._dofmap

    def on_boundary(self):
        """
        Returns a list of global numbering of all dofs on the boundary
        """
        b_dofs = []
        for i in range(len(self.mesh.vertices)):
            # For another FiniteElement, one would have to
            # keep a separate dof-map from the mesh
            if self.mesh.on_boundary(self.mesh.vertices[i]):
                b_dofs.append(i)
        return b_dofs

In [17]:
class Dikdörtgen():
    def __init__(self, L, H, nx, ny):
        
        x = np.linspace(0, L, nx+1)
        y = np.linspace(0, H, ny+1)
        ağ = np.meshgrid(x, y)

        noktalar = np.zeros(((nx+1)*(ny+1), 2), dtype=np.float32)
        hücreler = np.zeros((nx*ny, 4), dtype=np.int32)

        nokta = 0
        for iy in range(ny + 1):
            for ix in range(nx + 1):
                noktalar[nokta, :] = x[ix], y[iy]
                nokta += 1

        hücre = 0
        for iy in range(ny):
            for ix in range(nx):
                v0 = iy*(nx + 1) + ix
                v1 = v0 + 1
                v2 = v0 + nx+1
                v3 = v1 + nx+1
                hücreler[hücre, :] = v0, v1, v3, v2
                hücre += 1

        self.hücreler = hücreler
        self.noktalar = noktalar
        self.ağ = ağ

    def on_boundary(self, x):
        """ Checking if a coordinate is on the boundary of the mesh """
        return (np.isclose(x[0], 0) or np.isclose(x[1], 0)
                or np.isclose(x[0], L) or np.isclose(x[1], H))

In [43]:
nx, ny = 2, 2
plaka = Dikdörtgen(L,H,nx,ny)
V = FonksiyonUzayı(plaka)
V.dofmap()(3,2)

8

In [41]:
plaka.hücreler

array([[0, 1, 4, 3],
       [1, 2, 5, 4],
       [3, 4, 7, 6],
       [4, 5, 8, 7]], dtype=int32)